In [1]:
#import tensorflow as tf
#import tensorflow.keras as keras

import keras
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import glob
import qgrid
import cv2

#train_data_dir = 'C:/devenv/datasets/malaria_hero/images'
train_data_dir = 'C:\SAP project'
labels = ['Uninfected','Parasitized']
#valid_data_dir = 'f1_mal/valid'

def extract_filenames_and_labels():
    result = []
    for idx, folder_name in enumerate(labels):
        files = glob.glob(train_data_dir + '/' + folder_name + '/*.png') 
        for filename in files:
            result.append((filename , idx))
            
    return result
    
    
dataset = extract_filenames_and_labels()
print(len(dataset))

df = pd.DataFrame(dataset,columns=['filename','label'])
df.to_csv('C:\Sap project\images.csv', sep='\t')

print('Uninfected: ',len(df[df['label'] == 0]))
print('Infected: ',len(df[df['label'] == 1]))
qgrid.show_grid(df)
#print(images)

# TODO save the dataset to csv/excel to have reproducible results and one-one mapping to the feature vectors np matrix
# DONE

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


27558
Uninfected:  13779
Infected:  13779


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [2]:
sizes = []
max_width = 385
max_height = 394

for filename, label in dataset:
    img = cv2.imread(filename, cv2.IMREAD_COLOR)
    img_width = img.shape[0]
    img_height = img.shape[1]
    img = cv2.copyMakeBorder(img,0,max_width - img_width,0,max_height - img_height, cv2.BORDER_CONSTANT, value = 0)
    #print(img.shape, img.dtype)
    sizes.append((filename,img.shape[0],img.shape[1],label))
    cv2.imwrite(filename,img)

images = [cv2.imread(filename,cv2.IMREAD_COLOR) for filename, label in dataset]
sizes_df = pd.DataFrame(sizes,columns=['filename','width','height','label'])
qgrid.show_grid(sizes_df)
#print(sizes_df)
# TODO pad to 385 394
# DONE

KeyboardInterrupt: 

In [21]:
# TODO rewrite this to keras, because it can accomodate arbitrary imput shape
# Hint use keras model.predict to pass an image np array and get the vector as an output
features = inception_v3(images)

steps = int(n_samples/batch_size)
print('Steps: ',steps)

data = np.zeros((n_samples,features.shape[1].value),dtype=np.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(steps):
        print(step,'\r',end='')
        f_data = sess.run(features)
        data[step*batch_size : step*batch_size+batch_size,:] = f_data
        if step % 100 == 0:
            print('Saving intermediate data \r', end='')
            np.save('C:/devenv/datasets/malaria_hero/features',data)            


print('Saving final data')
np.save('C:/devenv/datasets/celeba/features',data)

NameError: name 'inception_v3' is not defined

In [3]:
import keras
keras_iv3 = keras.applications.inception_v3.InceptionV3(include_top=False,pooling='max')
#keras_iv3.summary()

In [8]:
keras_iv3.input.shape

TensorShape([Dimension(None), Dimension(None), Dimension(None), Dimension(3)])

In [23]:
keras_iv3.output.shape

TensorShape([Dimension(None), Dimension(2048)])

In [4]:
number_batches = 10 #4593
print(number_batches)
def gen():
    images = np.zeros((6,385,394,3))
    br = 0
    for filename, idx in dataset:
        if br%6 == 0 and br != 0:
            yield images
            images = np.zeros((6,385,394,3))
        id = br%6
        images[id] = cv2.imread(filename, cv2.IMREAD_COLOR)
        br += 1
    yield images
def iterate():
    for image in gen():
        return image

mal_dataset = keras_iv3.predict_generator(gen(),steps = number_batches)
#print(output)

10


In [5]:
print(mal_dataset.shape)

(60, 2048)


In [6]:
from sklearn import datasets
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set(color_codes=True)

df = pd.DataFrame(mal_dataset)
sns.pairplot(df)

KeyboardInterrupt: 

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x0000022F8814BF28> (for post_execute):


ValueError: Image size of 368640x368640 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 368640x368640 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 368640x368640 with 3876 Axes>

In [15]:
pca = PCA(n_components = 2, whiten = False).fit(mal_dataset)
print('components:\n',pca.components_.T)
print('explained ratio:\n',pca.explained_variance_ratio_)

reduced = pca.transform(mal_dataset)

df = pd.DataFrame(reduced)
print(df)
sns.scatterplot(data = df)

components:
 [[ 0.0088621   0.00028455]
 [ 0.00767512  0.02840499]
 [-0.00199919  0.00327076]
 ...
 [ 0.01050636  0.02598792]
 [ 0.00026898 -0.00015286]
 [ 0.0004719   0.00052736]]
explained ratio:
 [0.3613376  0.14845006]
              0           1
0   -428.306580 -213.797897
1   -382.082733 -219.400085
2   -179.785355 -342.391846
3   1007.457397  250.065659
4    505.735138 -179.934540
5    987.942932   38.311394
6   -225.781860   22.008377
7    627.962463 -349.376465
8   -295.808411   24.221256
9   -104.376137 -480.694214
10  -269.526428 -147.086639
11  -299.123322 -167.130280
12    73.870483 -239.339661
13  1004.117493 -239.785751
14  -296.331909  -31.401573
15   599.807617  252.078812
16  -167.370529 -411.151001
17  -627.150757   77.081406
18   803.105713  -32.842506
19  -344.388885 -393.888153
20   -11.782044 -373.778778
21   382.153839   35.882030
22  -148.966537 -126.109505
23   771.753784  285.653534
24  -732.195923  394.826355
25  -822.014587  400.670959
26  -256.807953 -360.

AttributeError: module 'seaborn' has no attribute 'scatterplot'